лабораторная 6 — rag


подготовка окружения
эта ячейка нужна один раз на чистом окружении. если библиотеки уже стоят, можно пропустить.


In [79]:
# разворачиваем зависимости (нужны sentence-transformers, faiss и клиент openrouter)
!pip install -q sentence-transformers faiss-cpu openai tiktoken


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [80]:
import os
import re
from typing import List, Tuple

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import OpenAI


In [81]:
WIKI_PATH = "wiki.txt"
if not os.path.exists(WIKI_PATH):
    raise FileNotFoundError("Файл wiki.txt не найден рядом с ноутбуком")

with open(WIKI_PATH, "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"{len(raw_text):,}")
print(f"{len(raw_text.splitlines()):,}")


3,809
50


разбивка по блокам

In [82]:
def chunk_text(text: str) -> List[str]:
    lines = [ln.rstrip() for ln in text.split('\n')]
    chunks: List[str] = []
    buf: List[str] = []
    for ln in lines:
        if re.match(r'^\d+\.\s', ln) and buf:
            chunks.append('\n'.join([x for x in buf if x]))
            buf = [ln]
        else:
            buf.append(ln)
    if buf:
        chunks.append('\n'.join([x for x in buf if x]))
    return [c for c in chunks if c.strip()]


chunks = chunk_text(raw_text)
print(f"Число чанков: {len(chunks)}")
preview = min(3, len(chunks))
for i in range(preview):
    print(f"\n[пример чанка #{i+1}]\n{chunks[i]}\n")
if len(chunks) > preview:
    print(f"... и ещё {len(chunks) - preview} фрагментов")


Число чанков: 8

[пример чанка #1]
УТВЕРЖДЕНО
Верховным Алгоритмом
от 32 мартобря 2077 года
РЕГЛАМЕНТ ВНУТРЕННЕГО ТРУДОВОГО РАСПОРЯДКА
ООО «КИБЕР-ЧЕБУРЕКИ» (Версия 42.0-beta)


[пример чанка #2]
1. ОБЩИЕ ПОЛОЖЕНИЯ
1.1. Настоящий регламент регулирует поведение биологических и полубиологических единиц (далее — Сотрудники) в офисах и на орбитальных станциях Компании.
1.2. Каждый сотрудник обязан носить идентификационный чип в левом ухе. В случае потери уха в производственной дуэли, чип переносится в нос или иную выступающую часть тела.
1.3. Рабочий день начинается, когда индикатор биоритмов Директора загорается зеленым, и заканчивается, когда уровень кислорода в офисе падает до 15%.
1.4. Незнание законов робототехники не освобождает от ответственности перед лазерным шредером.


[пример чанка #3]
2. БЕЗОПАСНОСТЬ И ДОСТУП
2.1. При обнаружении в столовой робота-шпиона модели Т-800, запрещается вступать с ним в философские споры о смысле жизни. Следует немедленно предложить ему смазку WD-40 и

индексация

In [83]:
model_name = "cointegrated/rubert-tiny2"
embedder = SentenceTransformer(model_name)

embeddings = embedder.encode(
    chunks,
    convert_to_numpy=True,
    show_progress_bar=True,
    batch_size=64,
    normalize_embeddings=True,
)
embeddings = embeddings.astype("float32")

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)
print(f"FAISS-индекс готов: {index.ntotal} векторов")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS-индекс готов: 8 векторов


поиск кандидатов

In [84]:
def lexical_overlap_score(text: str, query_tokens: set) -> float:
    tokens = set(re.findall(r"\w+", text.lower()))
    if not query_tokens:
        return 0.0
    return len(tokens & query_tokens) / (len(query_tokens) + 1e-6)


def retrieve_candidates(query: str, top_k: int = 5, search_k: int = 12) -> List[Tuple[str, float, float]]:
    q_vec = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    q_vec = q_vec.astype("float32")
    sims, idxs = index.search(q_vec, search_k)
    q_tokens = set(re.findall(r"\w+", query.lower()))

    scored = []
    for sim, idx in zip(sims[0], idxs[0]):
        ctx = chunks[idx]
        overlap = lexical_overlap_score(ctx, q_tokens)
        combined = 0.7 * float(sim) + 0.3 * overlap
        scored.append((combined, float(sim), float(overlap), ctx))

    scored.sort(key=lambda x: x[0], reverse=True)
    return [(ctx, sim, overlap) for combined, sim, overlap, ctx in scored[:top_k]]


demo_query = "Когда разрешена работа из дома?"
print(f"\n=== проверка поиска: {demo_query} ===")
for i, (ctx, sim, ov) in enumerate(retrieve_candidates(demo_query, top_k=2), 1):
    snippet = ctx if len(ctx) < 600 else ctx[:600] + "…"
    print(f"\nкандидат #{i}\nsim={sim:.4f} | overlap={ov:.3f}\n{snippet}\n")



=== проверка поиска: Когда разрешена работа из дома? ===

кандидат #1
sim=0.5118 | overlap=0.800
5. УДАЛЕННАЯ РАБОТА (НЕЙРО-СВЯЗЬ)
5.1. Работа из дома разрешена только при наличии стабильного нейро-канала скоростью не менее 10 Терабит/с.
5.2. Если во время зум-колла у сотрудника на фоне появляется кот, кот обязан быть одет в корпоративный галстук. Коты без галстуков считаются посторонними агентами и подлежат оцифровке.
5.3. Запрещается выходить в астрал в рабочее время без письменного разрешения начальника отдела ментальной гигиены.


кандидат #2
sim=0.4507 | overlap=0.400
2. БЕЗОПАСНОСТЬ И ДОСТУП
2.1. При обнаружении в столовой робота-шпиона модели Т-800, запрещается вступать с ним в философские споры о смысле жизни. Следует немедленно предложить ему смазку WD-40 и вызвать IT-шамана.
2.2. Пароль от корпоративного Wi-Fi "Skynet_Guest" меняется каждый раз, когда курс биткоина падает более чем на 10%. Текущий пароль можно узнать, принеся жертву (пончик) сисадмину.
2.3. Категорически зап

In [ ]:
api_key = "pass"

client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)
DEFAULT_MODEL = os.getenv("OPENROUTER_MODEL", "amazon/nova-2-lite-v1:free")


def generate_answer(question: str, context_docs: List[str], model: str = DEFAULT_MODEL) -> str:
    context_text = "\n\n".join(context_docs)
    system_prompt = (
        "Ты — внимательный помощник корпоративной базы знаний. "
        "Используй только предоставленный контекст и не додумывай детали. "
        "Если информации нет, отвечай: 'Не могу ответить, в базе этого нет'. "
        "Пиши лаконично: один-два абзаца или короткий маркированный список."
    )
    user_prompt = f"Контекст:\n{context_text}\nВопрос:\n{question}"

    response = client.chat.completions.create(
        model=model,
        max_tokens=400,
        temperature=0.1,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )
    return response.choices[0].message.content


In [86]:
def answer_question(question: str, top_k: int = 4, model: str = DEFAULT_MODEL) -> str:
    candidates = retrieve_candidates(question, top_k=top_k)
    selected_contexts = [c for c, _, _ in candidates]

    print("\n=== топ кандидатов ===")
    if not candidates:
        print("ничего не найдено — попробуйте переформулировать запрос.")
    for i, (ctx, sim, ov) in enumerate(candidates, 1):
        snippet = ctx.replace("\n", " ")
        if len(snippet) > 400:
            snippet = snippet[:397] + "..."
        print(f"\n#{i} | sim={sim:.4f} | overlap={ov:.3f}\n{snippet}\n")

    answer = generate_answer(question, selected_contexts, model=model)
    print("\n=== ответ модели ===")
    print(answer)
    return answer


In [ ]:
sample_questions = [
    "Какой пароль от корпоративного wifi",
    "Какой девиз компании ?",
    "за сколько времени нужно подавать заявление об увольнении ?"
]

RUN_DEMO = True 

if RUN_DEMO:
    for q in sample_questions:
        print(f"\n===== {q} =====")
        answer_question(q, top_k=2)
else:
    print("Установите RUN_DEMO = True и перезапустите ячейку, чтобы протестировать пайплайн.")



===== Какой пароль от корпоративного wifi =====

=== топ кандидатов ===

#1 | sim=0.4952 | overlap=0.600
2. БЕЗОПАСНОСТЬ И ДОСТУП 2.1. При обнаружении в столовой робота-шпиона модели Т-800, запрещается вступать с ним в философские споры о смысле жизни. Следует немедленно предложить ему смазку WD-40 и вызвать IT-шамана. 2.2. Пароль от корпоративного Wi-Fi "Skynet_Guest" меняется каждый раз, когда курс биткоина падает более чем на 10%. Текущий пароль можно узнать, принеся жертву (пончик) сисадмину. ...


#2 | sim=0.4559 | overlap=0.200
1. ОБЩИЕ ПОЛОЖЕНИЯ 1.1. Настоящий регламент регулирует поведение биологических и полубиологических единиц (далее — Сотрудники) в офисах и на орбитальных станциях Компании. 1.2. Каждый сотрудник обязан носить идентификационный чип в левом ухе. В случае потери уха в производственной дуэли, чип переносится в нос или иную выступающую часть тела. 1.3. Рабочий день начинается, когда индикатор биоритм...


=== ответ модели ===
### Ответ:

Согласно разделу **2.2*